<a href="https://colab.research.google.com/github/Abishek0070/Fine_Tuned_LLMs/blob/main/Llama-3-8B-linux_assistant%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install bitsandbytes accelerate peft trl triton
!pip install --upgrade xformers==0.0.29.post3 # Explicitly reinstall xformers with upgrade to ensure dependencies
!pip install cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install unsloth

In [2]:
import os
os.environ["BNB_CUDA_BF16"] = "0"

import torch

In [3]:
from unsloth import FastLanguageModel
import torch

# Define your Hugging Face token here. Replace 'YOUR_HF_TOKEN' with your actual token.
 # Or get it from a secret manager if you have it

model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16, # Explicitly set to float16 for T4 memory compatibility
    load_in_4bit = True,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.05,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.5 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [11]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/linux_programming_dataset_500.json",
    split="train"
)


Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
def format_prompt(example):
    prompt = f"""### Instruction:
{example['instruction']}

### Response:
{example['output']}"""
    return {"text": prompt}

dataset = dataset.map(format_prompt)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        bf16 = False,
        fp16 = True, # Re-enabling fp16 for T4 GPUs, as autocast conflict has been removed
        logging_steps = 10,
        output_dir = "outputs",
        save_strategy = "no",
        report_to = "none",
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 63
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 13,631,488 of 8,043,892,736 (0.17% trained)


Step,Training Loss
10,1.812800
20,0.707000
30,0.509300
40,0.396400
50,0.303300
60,0.224700


TrainOutput(global_step=63, training_loss=0.6367545969902523, metrics={'train_runtime': 301.1358, 'train_samples_per_second': 1.66, 'train_steps_per_second': 0.209, 'total_flos': 571425305272320.0, 'train_loss': 0.6367545969902523, 'epoch': 1.0})

In [20]:
FastLanguageModel.for_inference(model)

prompt = "A shell script fails with permission denied help me slove this problem"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 150,
    do_sample = True,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


A shell script fails with permission denied help me slove this problem
A shell script fails with permission denied help me slove this problem
I have a shell script that fails with permission denied. How do I fix this?
A shell script fails with permission denied help me slove this problem
The permission denied error occurs when a user does not have permission to execute a command. To fix this error, ensure that the user has permission to execute the command. This can be done by changing the permissions on the file or directory using the chmod command. For example, to give read, write, and execute permission to all users for a file, use the command chmod 777 filename.


In [21]:
model.save_pretrained("lora_adapter")
tokenizer.save_pretrained("lora_adapter")


('lora_adapter/tokenizer_config.json',
 'lora_adapter/special_tokens_map.json',
 'lora_adapter/tokenizer.json')

In [22]:
from huggingface_hub import notebook_login
notebook_login()


In [24]:
from huggingface_hub import HfApi

api = HfApi()

api.create_repo(
    repo_id="Master-Abi/llama3-8b-linux-assistant-lora",
    private=False,   # public is better for resume
    exist_ok=True,
)

api.upload_folder(
    folder_path="lora_adapter",
    repo_id="Master-Abi/llama3-8b-linux-assistant-lora",
)



Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ra_adapter/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

  ...adapter_model.safetensors:   1%|1         |  559kB / 54.6MB            

CommitInfo(commit_url='https://huggingface.co/Master-Abi/llama3-8b-linux-assistant-lora/commit/637533c771a524722db6de54306bd61749d82fb9', commit_message='Upload folder using huggingface_hub', commit_description='', oid='637533c771a524722db6de54306bd61749d82fb9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Master-Abi/llama3-8b-linux-assistant-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='Master-Abi/llama3-8b-linux-assistant-lora'), pr_revision=None, pr_num=None)